# Hidden Number Problem

## Inicialização da classe
Foi implementada uma classe que permite a resolução de problemas de Hidden Number Problem (HNP). São nesta guardadas as variáveis necessárias para a resolução do problema, bem como a lógica do mesmo. Esta recebe três inteiros `p`, `k` e `l` na sua inicialização, com os quais efetua os seguintes passos:
1. Guardam-se as variáveis de inicialização.
2. Calcula e armazena-se o `lambda = 2^(k+1)` é utilizado na inicialização das matrizes `L` e `target`.
3. Inicializa-se o vetor `x` com `l` valores inteiros entre `[0..p - 1]`.
4. Inicializa-se a matriz `L` de dimensão `(l + 1) * (l + 1)`.   $${L}\;\equiv\; \left\lbrack \begin{array}{c|c} p\,\mathbf{I}_\ell & 0 \\\mathbf{x} & \lambda^{-1} \end{array} \right\rbrack$$ 
Como esta é originalmente uma matriz de racionais, os seus valores são também multiplicados por `lambda` de modo a torná-la uma matriz de inteiros: $${L}\;\equiv\; \left\lbrack \begin{array}{c|c} \lambda\,p\,\mathbf{I}_\ell & 0 \\\lambda\,\mathbf{x} & 1 \end{array}\right\rbrack$$

## Inicialização das matrizes de incógnitas
O HNP trata de determinar um inteiro `s` a partir dos vetores `target` e `w` de dimensão `l + 1`. Estes são derivadas a partir do valor `s` bem como do vetor `x`:
+ O vetor `target` obtém-se recorrendo à aplicação da função de inteiros `msb` ao produto de cada um dos inteiros de `x` com `s`, sendo o `l+1`-ésimo elemento igual a 0. A função `msb` é responsável por calcular um inteiro a partir dos `k` bits mais significativos de um dado valor. Como esta se trata originalmente de um vetor de racionais, cada elemento é posteriormente multiplicado por `lambda` de modo a torná-a uma matriz de inteiros.
+ O vetor `w` obtém-se recorrendo à aplicação da função de inteiros `quo` ao produto de cada um dos inteiros de `x` com `s`, sendo o `l+1`-ésimo elemento igual a `s`. A função `quo` é responsável por calcular a divisão inteira de um valor por `p`.

## Resolução do HNP
Para 
1. Calcula-se `w * L - target` e calcula-se a norma do vetor resultante a ordem `p`, que se define pelo máximo do cálculo `min(|x|, p - |x|)` sobre cada um dos valores do vetor resultante. 
2. Compara-se a norma com `p`. Caso esta seja menor ou igual a `p`, é possível concluir que o vetor `α ≡ w * L` é muito provavelmente o ponto do reticulado mais próximo de `target`. Caso contrário, não é possível discernir qual será o vetor mais próximo de `target`.
3. Calculando `α`, seria possível recuperar `s` através do produto do seu `l+1`-ésimo valor com `lambda`. No entanto, devido à configuração aplicada, necessita-se apenas extrair o `l+1`-ésimo valor sem quaisquer outros cálculo, dado que, relativo à configuração original, `α = (lambda * w) * (lambda * L) = lambda * (w * L)`. .

In [1]:
class HNP:
    def __init__(self, p, k, l):
        self.p = p
        self.k = k
        self.l = l
        self.x = [0] * l
        self.lam = 2**(k + 1)
        self.L = self.lam * p * matrix.identity(l+1)
        self.L[l, l] = 1
        for i in range(l):
            self.x[i] = int(ZZ.random_element(l))
            self.L[l,i] = self.lam * self.x[i]
    
    def target(self, s):
        t = [0] * (self.l + 1)
        for i in range(self.l):
            t[i] = hnp.lam * hnp.msb(hnp.x[i] * s)
        return t
    
    def w(self, s):
        w = [0] * (hnp.l + 1)
        for i in range(hnp.l):
            w[i] = -hnp.quo(int(self.x[i] * s))
        w[hnp.l] = s
        return w
        
    def msb(self, s):
        v = int(ZZ(s.bits()[-self.k::],2))
        return mod(v,self.p)

    def quo(self, s):
        return s//self.p
    
    def norm(self, matrix):
        m = -1
        for x in matrix:
            aux = min(abs(int(x)), self.p - abs(int(x)))
            if aux > m:
                m = aux
        return m
    
    def solve(self, t, w):
        val = (vector(w)*self.L-vector(t))
        
        norm = int(str(hnp.norm(val)))
        
        print("norm = " + str(norm))
        print("p = " + str(hnp.p))
        
        if norm > self.p:
            raise ValueError("Norma superior ao valor de p")
        
        alpha = vector(w) * self.L
        
        s = alpha[self.l]
        
        return s

# Exemplo de inicialização do problema

In [2]:
hnp = HNP(50,50,256)

s = ZZ.random_element(hnp.l)
while s == 0:
    s = ZZ.random_element(hnp.l)

t = hnp.target(s)
w = hnp.w(s)

# Exemplo de resolução do problema

In [3]:
%time solution = hnp.solve(t, w)

print("s = " + str(s))
print("solution = " + str(solution))
print("s = solution ? " + str(s == solution))

norm = 10
p = 50
CPU times: user 78 ms, sys: 156 ms, total: 234 ms
Wall time: 240 ms
s = 190
solution = 190
s = solution ? True
